# 5. Curve Interpolation

In [3]:
from fixedincome2025 import table

## What To Do With Missing Points? 

* When you compute the present value (PV) of your bond portfolio in practice, there will be missing points on the yield curve
* Treasury yield curve from [treasury.gov](https://home.treasury.gov/resource-center/data-chart-center/interest-rates/TextView?type=daily_treasury_yield_curve&field_tdr_date_value=2025): 

In [19]:
table('yc_09152025')

Date,1 Mo,1.5 Mo,2 Mo,3 Mo,4 Mo,6 Mo,1 Yr,2 Yr,3 Yr
09/15/2025,4.22%,4.21%,4.17%,4.06%,4.00%,3.81%,3.64%,3.54%,3.50%


* Imagine you have a Treasury bond that's 11M from its maturity, and the second-to-last coupon payment is 5M from now
* You need 5M point and 11M point to price the bond
* One also needs to take DCC into consideration when computing the year fractions ($T$ in the discount factor $e^{-TR(0, T)}$)

## A World of Curves

* Although we've been using the Treasury yield curve as our only example, in practice there are many other curves
    * Important ones are SOFR, Term SOFR, fed funds, LIBOR
    * SOFR is by far the most important one if you work in the rates business in a bank, more important than the Treasury yield curve
    * In the rates business, many curves are simply a spread curve plus the SOFR curve, spread from basis swap quotes
    * In a credit desk (trading credit projects including corporate bonds), corporate bond yield curves are a spread curve (known as the z-spread) plus the Treasury yield curve
* **Each curve has its own 6 representations**
* Below is how a SOFR curve should be interpolated

## Curve Interpolation Methods

* Curve interpolation is on the time-weighted zero rate $TR(0, T)$
* The interpolation method is
    * Linear interpolation in the short end
    * Cubic spline for the rest
* Cutoff is around 3Y, a decision more of an art than science

## Why Linear Interpolation on Time-Weighted Zero Rate? 

* Recall that
\begin{align*}
f(0, T) &= \frac{\partial }{\partial T} [TR(0, T)]
\end{align*}
* The reason people use linear interpolation on time-weighted zero rate in the short end is that we want its first derivative, the instantaneous forward curve ($f(0, T)$ for all $T$), to be a step function in the short end
* Linear interpolating function: Piecewise linear
* Step function: Piecewise constant
* The first derivative of a linear interpolating function is a step function

## Why Piecewise Constant Instantaneous Forward Curve? 

* Recall that
$$
L(0, T, T+\tau) = \frac{1}{\tau}\left(e^{\int_T^{T+\tau} f(0, u)\,du} - 1\right)
$$
* A variant of this formula is used to quote FRAs (and swaps)
* Recall the CME Term SOFR example

In [12]:
table('term_sofr')

,Term SOFR 1M,Term SOFR 3M,Term SOFR 6M,Term SOFR 12M
09-17-2025,4.13359 %,4.02304 %,3.85385 %,3.59516 %
09-16-2025,4.13584 %,4.02570 %,3.85447 %,3.60384 %
09-15-2025,4.14288 %,4.02330 %,3.84833 %,3.60702 %


* Think of $f(0, T) = L(0, T, T+\epsilon)$ for $\epsilon=$1d
* $f(0, T)$ is the market anticipated time-$T$ (in the future) daily SOFR fixings, which should stay constant between fed rate decisions
* The discontinuities are [FOMC meeting dates](https://www.federalreserve.gov/monetarypolicy/fomccalendars.htm)

## Linear Interpolation on Time-Weighted Zero Rate

* In conclusion, it makes sense for the $TR(0, T)$ curve to be a linear function between FOMC meetings dates in the short end
* The slope of $TR(0, T)$ at $T=T^*$ equals $f(0, T^*)$, which is the anticipated SOFR daily fixing level at $T^*$
* FOMC meeting schedule is not determined for 3Y out, why not just make the curve continuous and smooth? 

## Short End Curve Interpolation: Example I

* Assuming: 
    * Today's SOFR yield curve is as below
    * From the 2M point to the 3M point there will be no FOMC meetings
    * 30/360 DCC
* What should the yield be at the 2.5M point?

In [27]:
table('yc_sofr_example')

1 Mo,1.5 Mo,2 Mo,3 Mo,4 Mo,6 Mo,1 Yr,2 Yr,3 Yr
4.22%,4.21%,4.17%,4.06%,4.00%,3.81%,3.64%,3.54%,3.50%


* Between 2M and 3M, $TR(0, T)$ must be a linear function, so we have 
\begin{align*}
&\left(\frac{2}{12}\times 4.17\% + \frac{3}{12}\times 4.06\%\right)/2 = \frac{2.5}{12} \times R\\
\end{align*}
and hence $R = 4.104\%$

## Short End Curve Interpolation: Exercise I

* Assuming: 
    * Today's SOFR yield curve is as below
    * From the 2M point to the 3M point there will be no FOMC meetings
    * 30/360 DCC
* What should the yield be at the 2.333M point?

In [28]:
table('yc_sofr_exercise')

1 Mo,1.5 Mo,2 Mo,3 Mo,4 Mo,6 Mo,1 Yr,2 Yr,3 Yr
4.20%,4.19%,4.15%,4.04%,4.00%,3.81%,3.62%,3.51%,3.47%


## Short End Curve Interpolation: Example II

* Assuming: 
    * Today's SOFR yield curve is as below
    * From the 1.5M point to the 2M point there will be no FOMC meetings
    * From the 3M point to the 3.5M point there will be no FOMC meetings
    * There is an FOMC meeting at the 2.5M point where the consensus is that the Fed will cut its rate by 25 bps
    * SOFR daily rates moves at the same time by the same amount as the Fed target rate
    * 30/360 DCC
* What should the yield be at the 3.333M point?

In [27]:
table('yc_sofr_example')

1 Mo,1.5 Mo,2 Mo,3 Mo,4 Mo,6 Mo,1 Yr,2 Yr,3 Yr
4.22%,4.21%,4.17%,4.06%,4.00%,3.81%,3.64%,3.54%,3.50%


## Short End Curve Interpolation: Example II (Cont.)

In [27]:
table('yc_sofr_example')

1 Mo,1.5 Mo,2 Mo,3 Mo,4 Mo,6 Mo,1 Yr,2 Yr,3 Yr
4.22%,4.21%,4.17%,4.06%,4.00%,3.81%,3.64%,3.54%,3.50%


* Recall that the slope of $TR(0, T)$ in the interval [1.5M, 2M] is $f(0, T)$, the anticipated SOFR daily fixing level in that interval. Numerically, it's
\begin{align*}
\frac{2\times 4.17\% - 1.5\times 4.21\%}{0.5} = 4.05\%
\end{align*}

* Similarly, the slope of $TR(0, T)$ in the interval [3M, 3.5M] is the anticipated SOFR daily fixings in that interval, which is $4.05\% - 0.25\% = 3.8\%$

* The slope can also be written in terms of yield at the 3M point and the 3.3333M point as
\begin{align*}
\frac{3.3333\times R - 3\times 4.06}{0.3333}, 
\end{align*}
which we know should be $3.8\%$. Thus $R$ can be backed out to be $4.034\%$